In [ ]:
import sys
!{sys.executable} -m pip install git+https://github.com/arendjan/tmx-pico-aio.git@sensors
!{sys.executable} -m pip install nest_asyncio

Hardware setup:

![button](images/ldr_bb.png)

Pico connections:

|Pin # | type  | connection |
|-------|------|------------|
| 18   | GND   |            |
| 31   | GP26  |Analog input, ADC0    |
| 36   | 3V3   |            |



In [ ]:
import asyncio
import sys
import time
from tmx_pico_aio import tmx_pico_aio
import nest_asyncio
import struct
nest_asyncio.apply() # for jupyter notebook asyncio stuff

loop = asyncio.get_event_loop()

board = tmx_pico_aio.TmxPicoAio(loop=loop)

Add your callback, like printing or adding it to a list

In [ ]:
MAX_ADC = 2**12
R1 = 10_000 # other resistor value, 10k
MAX_V = 3.3
ldr_vcc = True # is the ldr connected to VCC or the constant resistor

def calculate_ldr_resistance(adc_val):
    volt = adc_val/MAX_ADC*MAX_V
    # ratio = R1 : R_ldr
    ratio = volt/MAX_V if ldr_vcc else 1-volt/MAX_V
    factor = (1/ratio -1)
    ldr_res = factor * R1
    # print(f"a: {adc_val}  v: {volt} r: {ratio} fac: {factor}, ldr: {ldr_res}")
    return ldr_res


async def the_callback(data): # only will be called when there is a change
    adc_val = data[2]
    print(calculate_ldr_resistance(adc_val) )

Add the sensor to the system

In [ ]:
async def addSensor(board):
    await asyncio.sleep(1)
    adc = 0 # pin GP26
    await board.set_pin_mode_analog_input(adc, differential=0, callback=the_callback)

loop.run_until_complete(addSensor(board))



Run for some time

In [ ]:
async def run_till_end(board):
    # when to stop?
    await asyncio.sleep(10)
    await board.shutdown() # Otherwise next start will give you errors and you need to restart the kernel
    await asyncio.sleep(1)
    print("done")

loop.run_until_complete(run_till_end(board))